In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
import networkx as nx


In [31]:
import sys
sys.path.append('../../code')

from metrics import get_hi_metrics
import min_vertex_k_cut

# Prepare dataset

In [6]:
train = pd.read_csv('../../data/hi/hiv/train_1.csv', index_col=0)
test = pd.read_csv('../../data/hi/hiv/test_1.csv', index_col=0)

train

,smiles,value
4,O=S(=O)(O)CCS(=O)(=O)O,0
21,CC(C)CCS(=O)(=O)O,0
90,O=S(=O)(O)CCO,0
106,O=S(=O)(O)CO,0
117,O=S(=O)(O)CCCCBr,0
...,...,...
40932,COC(=O)c1cc2cc3c(c(O)c2c(=O)o1)OC1(Oc2c(O)c4c(...,0
40973,CCCCC1C(OCOc2ccccc2)COC(=O)N1C(C)c1ccccc1,0
41024,CC(C)=CC1CC(C)C2CCC(C)C3C(=O)C(O)=C(C)C(=O)C123,0
41026,CCOC(=O)C12C(=O)C(C)CCC1C(C)CC2C=C(C)C,0


# Split train into train and val

In [10]:
smiles = train['smiles'].to_list()
threshold = 0.4

neighborhood_graph = min_vertex_k_cut.get_neighborhood_graph(smiles, threshold)
main_component, small_components = min_vertex_k_cut.get_main_component(neighborhood_graph)

old_nodes_to_new = dict(zip(main_component.nodes(), range(main_component.number_of_nodes())))
new_nodes_to_old = {v: k for k, v in old_nodes_to_new.items()}
main_component = nx.relabel_nodes(main_component, old_nodes_to_new)

[13:38:09] WARNING: not removing hydrogen atom without neighbors
[13:38:09] WARNING: not removing hydrogen atom without neighbors


In [11]:
coarsed_main_component, node_to_cluster = min_vertex_k_cut.coarse_graph(main_component, 0.4)

In [13]:
model = min_vertex_k_cut.train_test_split_connected_graph(coarsed_main_component, train_min_fraq=0.5, test_min_fraq=0.1, max_mip_gap=0.3)

Total molecules: 4486
Min train size 2243
Min test size 448
Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 3471 (-1486) rows, 1486 (0) columns and 8424 (-2972) elements
Clp1000I sum of infeasibilities 3.0458e-06 - average 8.77499e-10, 0 fixed columns
Coin0506I Presolve 3471 (0) rows, 1486 (0) columns and 8424 (0) elements
Clp0029I End of values pass after 1486 iterations
Clp0014I Perturbing problem by 0.001% of 0.61265822 - largest nonzero change 2.9985502e-05 ( 0.002395959%) - largest zero change 0
Clp0000I Optimal - objective value 4486
Clp0000I Optimal - objective value 4486
Clp0000I Optimal - objective value 4486
Coin0511I After Postsolve, objective 4486, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 4486 - 0 iterations time 0.242, Presolve 0.00, Idiot 0.24

Starting MIP optimization
Cgl0004I processed model has 3471 rows, 1486 co

In [17]:
split = min_vertex_k_cut.process_bisect_results(model, coarsed_main_component, main_component, node_to_cluster)

Molecules in train: 3964
Molecules in test: 520
Molecules lost: 2


In [18]:
first_idx = []
second_idx = []

for S_idx, partition in enumerate(split):
    G_idx = new_nodes_to_old[S_idx]
    if partition == 0:
        first_idx.append(G_idx)
    if partition == 1:
        second_idx.append(G_idx)

In [19]:
for component in small_components:
    i = np.argmin([len(first_idx), len(second_idx)])
    if i == 0:
        first_idx.extend(component)
    if i == 1:
        second_idx.extend(component)

In [20]:
print(len(first_idx))
print(len(second_idx))

7847
7847


In [21]:
part_first = train.iloc[first_idx]
part_second = train.iloc[second_idx]

## Hi split

In [27]:
from sklearn.metrics import average_precision_score

def run_gb_gridsearch(train_fps, val_fps, train_y, val_y):
    split_index = [-1] * len(train_fps) + [0] * len(val_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + val_fps
    y = train_y + val_y

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    gb = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(gb, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    gb = GradientBoostingClassifier(**best_params)
    gb.fit(train_fps, train_y)

    val_preds = gb.predict_proba(val_fps)[:, 1]
    val_metrics = average_precision_score(val_y, val_preds)
    return val_metrics


In [28]:
train_mols = [Chem.MolFromSmiles(x) for x in part_first['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

val_mols = [Chem.MolFromSmiles(x) for x in part_second['smiles']]
val_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in val_mols]

test_metrics = run_gb_gridsearch(train_morgan_fps, val_morgan_fps, part_first['value'].to_list(), part_second['value'].to_list())
print(test_metrics)

[14:04:02] WARNING: not removing hydrogen atom without neighbors
[14:04:02] WARNING: not removing hydrogen atom without neighbors


Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=1.0, max_depth=3, max_features=None, min_samples_leaf=5, min_samples_split=3, n_estimators=250, subsample=0.9;, score=0.057 total time=  54.7s
[CV 1/1] END learning_rate=0.1, max_depth=3, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=500, subsample=0.7;, score=0.052 total time= 1.4min
[CV 1/1] END learning_rate=0.3, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50, subsample=0.9;, score=0.032 total time=   6.9s
[CV 1/1] END learning_rate=0.7, max_depth=4, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=200, subsample=0.9;, score=0.040 total time=   8.5s
[CV 1/1] END learning_rate=0.1, max_depth=2, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150, subsample=1.0;, score=0.046 total time=  28.2s
[CV 1/1] END learning_rate=0.7, max_depth=2, max_features=sqrt, min_samples_leaf=5, min_sa

### Final Evaluation

In [29]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

gb = GradientBoostingClassifier(
    subsample=0.9,
    n_estimators=250,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features='sqrt',
    max_depth=4,
    learning_rate=0.01
)

gb.fit(train_morgan_fps, train['value'].to_list())
test_preds = gb.predict_proba(test_morgan_fps)[:, 1]
test_metrics = average_precision_score(test['value'], test_preds)
print(test_metrics)


[14:20:25] WARNING: not removing hydrogen atom without neighbors
[14:20:25] WARNING: not removing hydrogen atom without neighbors


0.08437590235381434


# Scaffold split

In [32]:
from rdkit.ML.Cluster import Butina
from numpy.random import default_rng


def butina_split(smiles: list[str], cutoff: float, seed: int, frac_train=0.8):
    """
    Select distinct molecules to train/test. Returns indices of the molecules in the smiles list.
    Adapted from DeepChem (https://deepchem.io/), but random seed is added.
    """

    mols = [Chem.MolFromSmiles(smile) for smile in smiles]
    fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in mols]

    dists = []
    nfps = len(fps)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
        dists.extend([1 - x for x in sims])
    scaffold_sets = Butina.ClusterData(dists, nfps, cutoff, isDistData=True)
    scaffold_sets = sorted(scaffold_sets, key=lambda x: -len(x))

    rng = default_rng(seed)
    rng.shuffle(scaffold_sets)

    train_cutoff = frac_train * len(smiles)
    train_inds = []
    test_inds = []

    for scaffold_set in scaffold_sets:
        if len(train_inds) + len(scaffold_set) > train_cutoff:
            test_inds += scaffold_set
        else:
            train_inds += scaffold_set
    return train_inds, test_inds

In [33]:
train_idx, val_idx = butina_split(train['smiles'].to_list(), cutoff=0.5, seed=123, frac_train=0.5)

[14:24:31] WARNING: not removing hydrogen atom without neighbors
[14:24:31] WARNING: not removing hydrogen atom without neighbors


In [36]:
part_first = train.iloc[train_idx]
part_second = train.iloc[val_idx]

In [37]:
from sklearn.metrics import average_precision_score

def run_gb_gridsearch(train_fps, val_fps, train_y, val_y):
    split_index = [-1] * len(train_fps) + [0] * len(val_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + val_fps
    y = train_y + val_y

    params = {
    'n_estimators': [10, 50, 100, 150, 200, 250, 500],
    'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 1.0],
    'subsample': [0.4, 0.7, 0.9, 1.0],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
    'max_features': [None, 'sqrt']
    }
    gb = GradientBoostingClassifier()

    grid_search = RandomizedSearchCV(gb, params, cv=pds, n_iter=30, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    gb = GradientBoostingClassifier(**best_params)
    gb.fit(train_fps, train_y)

    val_preds = gb.predict_proba(val_fps)[:, 1]
    val_metrics = average_precision_score(val_y, val_preds)
    return val_metrics


In [38]:
train_mols = [Chem.MolFromSmiles(x) for x in part_first['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

val_mols = [Chem.MolFromSmiles(x) for x in part_second['smiles']]
val_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in val_mols]

test_metrics = run_gb_gridsearch(train_morgan_fps, val_morgan_fps, part_first['value'].to_list(), part_second['value'].to_list())
print(test_metrics)

[14:27:16] WARNING: not removing hydrogen atom without neighbors
[14:27:16] WARNING: not removing hydrogen atom without neighbors


Fitting 1 folds for each of 30 candidates, totalling 30 fits
[CV 1/1] END learning_rate=0.5, max_depth=2, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=200, subsample=0.9;, score=0.100 total time=   7.4s
[CV 1/1] END learning_rate=0.3, max_depth=4, max_features=None, min_samples_leaf=3, min_samples_split=2, n_estimators=10, subsample=0.9;, score=0.078 total time=   9.0s
[CV 1/1] END learning_rate=0.3, max_depth=4, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=250, subsample=1.0;, score=0.172 total time=   9.1s
[CV 1/1] END learning_rate=1.0, max_depth=4, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200, subsample=0.4;, score=0.034 total time=   7.6s
[CV 1/1] END learning_rate=1.0, max_depth=4, max_features=None, min_samples_leaf=5, min_samples_split=2, n_estimators=250, subsample=0.4;, score=0.044 total time=  31.4s
[CV 1/1] END learning_rate=0.01, max_depth=4, max_features=None, min_samples_leaf=1, min_s

### Final Evaluation

In [39]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

gb = GradientBoostingClassifier(
    subsample=0.9,
    n_estimators=100,
    min_samples_split=7,
    min_samples_leaf=5,
    max_features='sqrt',
    max_depth=4,
    learning_rate=0.01
)

gb.fit(train_morgan_fps, train['value'].to_list())
test_preds = gb.predict_proba(test_morgan_fps)[:, 1]
test_metrics = average_precision_score(test['value'], test_preds)
print(test_metrics)


[16:10:23] WARNING: not removing hydrogen atom without neighbors
[16:10:23] WARNING: not removing hydrogen atom without neighbors


0.07843671137825257
